# **Synthetic English Number Dataset Generation Scripts**

## **Script 1: Basic Augmentation (Optimal)**
The first commented block generates a digit dataset (0-9) with basic augmentations:
- **Random rotation** (-10 to 10 degrees)
- **Gaussian blur** (simulates out-of-focus camera)
- **Noise addition** (simulates sensor grain)
- **Erosion/Dilation** (simulates ink bleeding or thin print)

Uses a single font (Arial) and creates 200 samples per digit.

## **Script 2: Advanced Defects (Non-Optimal)**
The second commented block generates a more heavily augmented dataset with:
- **Multiple fonts** (Arial, Sitka) for style variation
- **Perspective warping** (simulates camera angle distortion)
- **Character breaking** (scratches and missing chunks)
- **Shadow simulation** (uneven lighting gradients)
- **Salt & pepper noise** (random black/white pixels)
- **Gaussian blur**

Creates 400 samples per digit but may over-augment, making digits harder to recognize (hence "non-optimal").

In [1]:
import os
import cv2
import numpy as np
import random
from PIL import Image, ImageDraw, ImageFont

# --- CONFIGURATION ---
OUTPUT_DIR = "dataset"
SAMPLES_PER_DIGIT = 200 # How many images per number to generate
IMG_SIZE = (64, 64)      # Canvas size (larger than target to allow rotation/cropping)
FONT_PATH = "arial.ttf"  # <--- COPY A FONT FILE HERE!
FONT_SIZE = 45

# Create folders 0-9
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
for i in range(10):
    os.makedirs(os.path.join(OUTPUT_DIR, str(i)), exist_ok=True)

def apply_augmentations(img_pil):
    """
    Takes a clean PIL image and ruins it to look like a scanned ID.
    """
    # Convert to NumPy for OpenCV processing
    img = np.array(img_pil) 
    
    # 1. Random Rotation (-10 to 10 degrees)
    angle = random.uniform(-10, 10)
    h, w = img.shape
    M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
    img = cv2.warpAffine(img, M, (w, h), borderValue=0) # Black border

    # 2. Gaussian Blur (Simulate out-of-focus camera)
    if random.random() > 0.5:
        k = random.choice([3, 5])
        img = cv2.GaussianBlur(img, (k, k), 0)

    # 3. Noise (Simulate sensor grain)
    noise = np.random.randint(0, 50, (h, w), dtype='uint8')
    # Add noise only to non-black areas mostly, or just add it overall
    img = cv2.add(img, noise)

    # 4. Erosion/Dilation (Simulate ink bleeding or thin print)
    if random.random() > 0.5:
        kernel = np.ones((2,2), np.uint8)
        if random.choice([True, False]):
            img = cv2.erode(img, kernel, iterations=1)
        else:
            img = cv2.dilate(img, kernel, iterations=1)

    return img

def generate_dataset():
    print(f"Generating {SAMPLES_PER_DIGIT} images per digit...")
    
    try:
        font = ImageFont.truetype(FONT_PATH, FONT_SIZE)
    except IOError:
        print("ERROR: Font file not found! Please put 'arial.ttf' in this folder.")
        return

    for digit in range(10):
        print(f"Processing digit: {digit}")
        for i in range(SAMPLES_PER_DIGIT):
            # 1. Create a blank black image
            img_pil = Image.new('L', IMG_SIZE, color=0)
            draw = ImageDraw.Draw(img_pil)
            
            # 2. Draw the digit in white centered(ish)
            # We add random offset so the number isn't always perfectly in the middle
            text = str(digit)
            # Get text bounding box to center it
            bbox = draw.textbbox((0, 0), text, font=font)
            text_w = bbox[2] - bbox[0]
            text_h = bbox[3] - bbox[1]
            
            x = (IMG_SIZE[0] - text_w) / 2 + random.randint(-5, 5)
            y = (IMG_SIZE[1] - text_h) / 2 + random.randint(-5, 5)
            
            draw.text((x, y), text, font=font, fill=255)
            
            # 3. Apply the "Reality" effects
            final_img = apply_augmentations(img_pil)
            
            # 4. Save
            save_path = os.path.join(OUTPUT_DIR, str(digit), f"{digit}_{i}.png")
            cv2.imwrite(save_path, final_img)

    print("Done! You now have a dataset.")

if __name__ == "__main__":
    generate_dataset()

Generating 200 images per digit...
Processing digit: 0
Processing digit: 1
Processing digit: 2
Processing digit: 3
Processing digit: 4
Processing digit: 5
Processing digit: 6
Processing digit: 7
Processing digit: 8
Processing digit: 9
Done! You now have a dataset.


In [ ]:
import os
import cv2
import numpy as np
import random
from PIL import Image, ImageDraw, ImageFont

# --- CONFIGURATION ---
OUTPUT_DIR = "dataset_mixed"
SAMPLES_PER_DIGIT = 400  # Total images per digit
IMG_SIZE = (64, 64)

# List your fonts here. Ensure these files are in the folder!
FONTS = ["arial.ttf", "Sitka.ttc"] 
FONT_SIZE = 45

# Create output folders
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
for i in range(10):
    os.makedirs(os.path.join(OUTPUT_DIR, str(i)), exist_ok=True)

def add_shadow(img):
    """Simulates uneven lighting."""
    h, w = img.shape
    top_left = random.uniform(0.5, 1.0)
    bot_right = random.uniform(0.5, 1.0)
    X, Y = np.meshgrid(np.arange(w), np.arange(h))
    mask = top_left + (bot_right - top_left) * (X / w)
    img = img.astype('float32') * mask
    return img.astype('uint8')

def break_character(img):
    """Simulates scratches and missing chunks."""
    h, w = img.shape
    
    # Random scratch line
    if random.random() > 0.5:
        num_scratches = random.randint(1, 3)
        for _ in range(num_scratches):
            x1, y1 = random.randint(0, w), random.randint(0, h)
            x2, y2 = random.randint(0, w), random.randint(0, h)
            cv2.line(img, (x1, y1), (x2, y2), 0, random.randint(1, 3))

    # Random noise chunks missing
    if random.random() > 0.5:
        noise = np.zeros((h, w), dtype='uint8')
        cv2.randn(noise, 0, 255)
        _, holes = cv2.threshold(noise, 200, 255, cv2.THRESH_BINARY)
        img = cv2.subtract(img, holes)
        
    return img

def apply_defects(img_pil):
    img = np.array(img_pil)
    
    # 1. Perspective Warp
    h, w = img.shape
    src_points = np.float32([[0,0], [w,0], [0,h], [w,h]])
    dst_points = np.float32([
        [random.randint(0, 5), random.randint(0, 5)],
        [w - random.randint(0, 5), random.randint(0, 5)],
        [random.randint(0, 5), h - random.randint(0, 5)],
        [w - random.randint(0, 5), h - random.randint(0, 5)]
    ])
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    img = cv2.warpPerspective(img, M, (w, h))

    # 2. Break Character
    img = break_character(img)

    # 3. Blur
    if random.random() > 0.3:
        k = random.choice([3, 5])
        img = cv2.GaussianBlur(img, (k, k), 0)

    # 4. Shadow
    if random.random() > 0.4:
        img = add_shadow(img)

    # 5. Salt & Pepper Noise
    noise_prob = 0.02
    thres = 1 - noise_prob
    rdn = np.random.random(img.shape)
    img[rdn < noise_prob] = 0
    img[rdn > thres] = 255

    return img

def generate_dataset():
    print(f"Generating {SAMPLES_PER_DIGIT} images per digit using fonts: {FONTS}")
    
    # Verify fonts exist
    loaded_fonts = []
    for f_name in FONTS:
        try:
            loaded_fonts.append(ImageFont.truetype(f_name, FONT_SIZE))
        except IOError:
            print(f"WARNING: Could not load {f_name}. Skipping.")
    
    if not loaded_fonts:
        print("ERROR: No fonts found! Please copy .ttf files to this folder.")
        return

    for digit in range(10):
        print(f"Processing digit: {digit}")
        for i in range(SAMPLES_PER_DIGIT):
            # 1. Randomly choose a font
            font = random.choice(loaded_fonts)
            
            # 2. Draw Text
            img_pil = Image.new('L', IMG_SIZE, color=0)
            draw = ImageDraw.Draw(img_pil)
            
            text = str(digit)
            bbox = draw.textbbox((0, 0), text, font=font)
            text_w, text_h = bbox[2] - bbox[0], bbox[3] - bbox[1]
            x = (IMG_SIZE[0] - text_w) / 2 + random.randint(-8, 8)
            y = (IMG_SIZE[1] - text_h) / 2 + random.randint(-8, 8)
            
            draw.text((x, y), text, font=font, fill=255)
            
            # 3. Apply Defects
            final_img = apply_defects(img_pil)
            
            # 4. Save
            save_path = os.path.join(OUTPUT_DIR, str(digit), f"{digit}_{i}.png")
            cv2.imwrite(save_path, final_img)

    print(f"Done! Created {SAMPLES_PER_DIGIT * 10} images in '{OUTPUT_DIR}'")

if __name__ == "__main__":
    generate_dataset()